In [40]:
import pandas as pd
import numpy as np
import os
import random
import base64

from pathlib import Path

base_path = Path.cwd().parent

In [41]:
base_path

WindowsPath('D:/Documents/Data Science Projects/2025 Hackathon_Mkt_SQL_Agent')

In [42]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

In [43]:
# Define the date range for event_date
start_dt = '2025-01-01'
end_dt = '2025-03-31'
date_range = pd.date_range(start=start_dt, end=end_dt, freq="D")

# Define possible values for categorical columns
campaign_products = ["Auto Insurance", "Homeowners", "Renters", "Credit Card", "Deposits", "Consumer Loans"]
campaign_cosas = {"Auto Insurance": "P&C", "Homeowners": "P&C", "Renters": "P&C",
                  "Credit Card": "Bank", "Deposits": "Bank", "Consumer Loans": "Bank"}
conversion_channels = ["Internet", "Mobile", "Offline"]
mkt_channels = ['Paid Display', 'Paid Search', 'Email', 'Paid Social', 'Direct Mail']

num_camps = 2 # number of campaigns for each product / cosa / channel

campaign_names_chnl = {}

for chnl in mkt_channels:
    campaign_names_chnl[chnl] = []
    for prod in campaign_products:
        for i in range(1, num_camps + 1):
            # Create campaign names for each product 
            campaign_name = f"{campaign_cosas[prod]}_{prod}_{chnl}_Campaign_{i}" 
            campaign_names_chnl[chnl].append(campaign_name)

In [44]:
campaign_names_chnl

{'Paid Display': ['P&C_Auto Insurance_Paid Display_Campaign_1',
  'P&C_Auto Insurance_Paid Display_Campaign_2',
  'P&C_Homeowners_Paid Display_Campaign_1',
  'P&C_Homeowners_Paid Display_Campaign_2',
  'P&C_Renters_Paid Display_Campaign_1',
  'P&C_Renters_Paid Display_Campaign_2',
  'Bank_Credit Card_Paid Display_Campaign_1',
  'Bank_Credit Card_Paid Display_Campaign_2',
  'Bank_Deposits_Paid Display_Campaign_1',
  'Bank_Deposits_Paid Display_Campaign_2',
  'Bank_Consumer Loans_Paid Display_Campaign_1',
  'Bank_Consumer Loans_Paid Display_Campaign_2'],
 'Paid Search': ['P&C_Auto Insurance_Paid Search_Campaign_1',
  'P&C_Auto Insurance_Paid Search_Campaign_2',
  'P&C_Homeowners_Paid Search_Campaign_1',
  'P&C_Homeowners_Paid Search_Campaign_2',
  'P&C_Renters_Paid Search_Campaign_1',
  'P&C_Renters_Paid Search_Campaign_2',
  'Bank_Credit Card_Paid Search_Campaign_1',
  'Bank_Credit Card_Paid Search_Campaign_2',
  'Bank_Deposits_Paid Search_Campaign_1',
  'Bank_Deposits_Paid Search_Campa

In [45]:
#Tables
# campaign taxonomy
# channel peformance
# --- One table for each channel - Paid Search, Paid Display, Paid Social
#   - Columns - event_date, campaign_product, campaign_cosa, converted_product, converted_cosa, campaign_nm, conversion_channel_nm, spend_amt, 
#   -         - quote_start_qty, quote_complete_qty, app_start_qty, app_complete_qty, prod_acq_qty, impression_qty, click_qty, campaign_funding_source      

# Function to generate a single dataset
def generate_agg_data(mkt_channel, campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl):
    # This function generates a dataset that mimics a last touch attribution aggregated data set
    # across all campaigns for a specific marketing channel.
    
    campaign_names = campaign_names_chnl[mkt_channel]

    data = []
    
    for date in date_range:
        for campaign in campaign_names:
            for camp_prod in campaign_products:
                for conv_prod in campaign_products:
                    for chnl in conversion_channels:

                        # Randomly select campaign_product and ensure campaign_cosa matches
                        campaign_product = camp_prod #random.choice(campaign_products)
                        campaign_cosa = campaign_cosas[campaign_product]
                        campaign_funding_source = f"{campaign_cosa} {campaign_product}"
                        
                        # Randomly select converted_product and ensure converted_cosa matches
                        converted_product = conv_prod #random.choice(campaign_products)
                        converted_cosa = campaign_cosas[converted_product]
                        
                        # Generate random values for other columns
                        conversion_channel = chnl # random.choice(conversion_channels)

                        # TODO: Create a function that generates the conversion quantities based on the impression / click qty (???? -- Future work if want output to look reasonable...)
                        # TODO: Create a function that also changes conversion quantities based on the conversion channel and product - to weight some products 
                        #         towards different conversion channels.
                        # TODO: Create a function that generates the impressions/clicks based on spend 
                        
                        impression_qty = random.randint(1000, 20000)
                        click_qty = random.randint(50, 200)
                        spend_amt = round(random.uniform(2000, 10000), 2)
                        quote_start_qty = random.randint(40, 150)
                        quote_complete_qty = random.randint(8, min(100, quote_start_qty))
                        app_start_qty = random.randint(5, min(80, quote_complete_qty))
                        app_complete_qty = random.randint(4, min(75, app_start_qty))
                        prod_acq_qty = random.randint(0, min(50, app_complete_qty))
                    
                        # Append the row to the dataset
                        data.append([
                            date, campaign_product, campaign_cosa, converted_product, converted_cosa,
                            campaign, conversion_channel, spend_amt, quote_start_qty, quote_complete_qty,
                            app_start_qty, app_complete_qty, prod_acq_qty, impression_qty, click_qty,
                            campaign_funding_source
                        ])
        
    # Create a DataFrame
    columns = [
        "event_date", "campaign_product", "campaign_cosa", "converted_product", "converted_cosa",
        "campaign_nm", "conversion_channel_nm", "spend_amt", "quote_start_qty", "quote_complete_qty",
        "app_start_qty", "app_complete_qty", "prod_acq_qty", "impression_qty", "click_qty",
        "campaign_funding_source"
    ]
    return pd.DataFrame(data, columns=columns)



In [46]:
# Generate the three datasets
Paid_Display = generate_agg_data('Paid Display', campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl)
Paid_Social = generate_agg_data('Paid Social', campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl)
Paid_Search = generate_agg_data('Paid Search', campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl)
Email = generate_agg_data('Email', campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl)
Direct_Mail = generate_agg_data('Direct Mail', campaign_products, campaign_cosas, conversion_channels, date_range, campaign_names_chnl)

# Example: Display the first few rows of one dataset
print(Paid_Display.head())

  event_date campaign_product campaign_cosa converted_product converted_cosa  \
0 2025-01-01   Auto Insurance           P&C    Auto Insurance            P&C   
1 2025-01-01   Auto Insurance           P&C    Auto Insurance            P&C   
2 2025-01-01   Auto Insurance           P&C    Auto Insurance            P&C   
3 2025-01-01   Auto Insurance           P&C        Homeowners            P&C   
4 2025-01-01   Auto Insurance           P&C        Homeowners            P&C   

                                  campaign_nm conversion_channel_nm  \
0  P&C_Auto Insurance_Paid Display_Campaign_1              Internet   
1  P&C_Auto Insurance_Paid Display_Campaign_1                Mobile   
2  P&C_Auto Insurance_Paid Display_Campaign_1               Offline   
3  P&C_Auto Insurance_Paid Display_Campaign_1              Internet   
4  P&C_Auto Insurance_Paid Display_Campaign_1                Mobile   

   spend_amt  quote_start_qty  quote_complete_qty  app_start_qty  \
0    7932.40            

In [47]:
len(Paid_Display)

116640

In [48]:
Paid_Display.to_csv(base_path / "data" / "Paid_Display_toy_data.csv", index=False)
Paid_Social.to_csv(base_path / "data" / "Paid_Social_toy_data.csv", index=False)
Paid_Search.to_csv(base_path / "data" / "Paid_Search_toy_data.csv", index=False)
Email.to_csv(base_path / "data" / "Email_toy_data.csv", index=False)
Direct_Mail.to_csv(base_path / "data" / "Direct_Mail_toy_data.csv", index=False)

In [49]:
#generate toy member data file / table

# Function to generate member data
def generate_member_data(num_members=1000):
    member_data = []
    for i in range(num_members):
        #Generate a random member number - Base64 hashed
        random_number = random.randint(10000000, 99999999)
        # Convert to Base64
        member_id = base64.b64encode(str(random_number).encode()).decode()
        age_grp = random.choice(["18-24", "25-34", "35-44", "45-54", "55-64", "65+"])
        marital_status = random.choice(["Single", "Married", "Divorced", "Widowed"])
        military_status = random.choice(["Active", "Separated","Retired","Spouse","Child", "None"])
        member_status = random.choice(["Active", "Inactive"])
        
        if member_status == "Inactive":
            active_pnc = False
            active_bank = False
            active_life = False
        else: 
            active_pnc = random.choice([True, False])
            active_bank = random.choice([True, False])
            active_life = random.choice([True, False])
        if active_pnc:
            active_auto_insurance = random.choice([True, False])
            active_homeowners = random.choice([True, False])
            if active_homeowners:
                active_renters = False
            else:
                active_renters = random.choice([True, False])
        else:
            active_auto_insurance = False
            active_homeowners = False
            active_renters = False
        if active_bank:
            active_credit_card = random.choice([True, False])
            active_deposits = random.choice([True, False])
        else:
            active_credit_card = False
            active_deposits = False
        if active_life:
            active_life_insurance = random.choice([True, False])
        else:
            active_life_insurance = False

        geo_city = random.choice(["New York", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia","San Antonio", "San Diego", "Dallas", "San Jose", "Austin"])

        member_data.append([member_id, age_grp, marital_status, military_status, member_status,
                            active_pnc, active_bank, active_life, active_credit_card,
                            active_deposits, active_auto_insurance, active_homeowners,
                            active_renters, active_life_insurance,geo_city])
    
    columns = ["member_id", "age_grp", "marital_status", "military_status", "member_status",
               "active_pnc", "active_bank", "active_life", "active_credit_card",
               "active_deposits", "active_auto_insurance", "active_homeowners",
               "active_renters", "active_life_insurance","geo_city"]
    return pd.DataFrame(member_data, columns=columns)

In [50]:
mbr_data = generate_member_data(10000)

In [51]:
mbr_data.head()

,member_id,age_grp,marital_status,military_status,member_status,active_pnc,active_bank,active_life,active_credit_card,active_deposits,active_auto_insurance,active_homeowners,active_renters,active_life_insurance,geo_city
0,NjY0MDk3MDk=,55-64,Married,Spouse,Inactive,False,False,False,False,False,False,False,False,False,San Antonio
1,OTY5NjU1NjU=,45-54,Married,Active,Inactive,False,False,False,False,False,False,False,False,False,San Diego
2,MzAyMzU3Njc=,65+,Married,Separated,Active,True,True,False,True,True,False,True,False,False,Philadelphia
3,MzMwNTA0MDI=,18-24,Divorced,Active,Active,True,False,True,False,False,True,True,False,True,San Jose
4,MTM1MjI0NDg=,18-24,Single,Active,Inactive,False,False,False,False,False,False,False,False,False,Chicago


In [52]:
mbr_data.to_csv(base_path / "data" / "member_data_toy_data.csv", index=False)

In [53]:
Paid_Display.columns

Index(['event_date', 'campaign_product', 'campaign_cosa', 'converted_product',
       'converted_cosa', 'campaign_nm', 'conversion_channel_nm', 'spend_amt',
       'quote_start_qty', 'quote_complete_qty', 'app_start_qty',
       'app_complete_qty', 'prod_acq_qty', 'impression_qty', 'click_qty',
       'campaign_funding_source'],
      dtype='object')

In [54]:
def generate_lta_data(start_dt, end_dt, mbr_data, agg_data, mkt_channel_nm, campaign_names_chnl,campaign_products, conversion_channels):
    # this function generates a toy last touch attribution data set for a given date range
    # Each row represents a "conversion" event for a member

    # The columns are: date, member_id, conversion_type, conversion_channel, product, campaign_name, mkt_channel
    # The conversion_type is one of: quote_start, quote_complete, app_start, app_complete, prod_acq
    # The conversion_channel is one of: Internet, Mobile, Offline
    # The product is one of: Auto Insurance, Homeowners, Renters, Credit Card, Deposits, Consumer Loans
    
    #initialize various lists
    date_range = pd.date_range(start=start_dt, end=end_dt, freq="D")
    event_data = []

    campaign_names = campaign_names_chnl[mkt_channel_nm]

    #Create a list to hold all combinations of parameters
    param_combinations = [(date, campaign, camp_prod, conv_prod, chnl) for date in date_range for campaign in campaign_names
                         for camp_prod in campaign_products for conv_prod in campaign_products for chnl in conversion_channels
                         if campaign.split('_')[1] == camp_prod]

    for date, campaign, camp_prod, conv_prod, chnl in param_combinations:
        # Filter agg_data based on the current combination of parameters
        camp_data = agg_data[
        (agg_data['event_date'] == date) &
        (agg_data['campaign_nm'] == campaign) &
        (agg_data['campaign_product'] == camp_prod) &
        (agg_data['conversion_channel_nm'] == chnl) &
        (agg_data['converted_product'] == conv_prod)
        ]
    
        # Iterate over the filtered agg_data to Get the number of conversions for each conversion type
        for index, row in camp_data.iterrows():
            quote_start_qty = row['quote_start_qty']
            quote_complete_qty = row['quote_complete_qty']
            app_start_qty = row['app_start_qty']
            app_complete_qty = row['app_complete_qty']
            prod_acq_qty = row['prod_acq_qty']

            #Create lists of member_ids for each conversion type
            quote_start_members = mbr_data.sample(n = quote_start_qty, replace = True)['member_id'].tolist() if quote_start_qty > 0 else []
            quote_complete_members = mbr_data.sample(n = quote_complete_qty, replace = True)['member_id'].tolist() if quote_complete_qty > 0 else []
            app_start_members = mbr_data.sample(n = app_start_qty, replace = True)['member_id'].tolist() if app_start_qty > 0 else []
            app_complete_members = mbr_data.sample(n = app_complete_qty, replace = True)['member_id'].tolist() if app_complete_qty > 0 else []
            prod_acq_members = mbr_data.sample(n = prod_acq_qty, replace = True)['member_id'].tolist() if prod_acq_qty > 0 else []
            
            # Extend the event_data list with the new events
            event_data.extend([(date, member_id, mkt_channel_nm, campaign, camp_prod, conv_prod, chnl, "quote_start") for member_id in quote_start_members])
            event_data.extend([(date, member_id, mkt_channel_nm, campaign, camp_prod, conv_prod, chnl, "quote_complete") for member_id in quote_complete_members])
            event_data.extend([(date, member_id, mkt_channel_nm, campaign, camp_prod, conv_prod, chnl, "app_start") for member_id in app_start_members])
            event_data.extend([(date, member_id, mkt_channel_nm, campaign, camp_prod, conv_prod, chnl, "app_complete") for member_id in app_complete_members])
            event_data.extend([(date, member_id, mkt_channel_nm, campaign, camp_prod, conv_prod, chnl, "prod_acq") for member_id in prod_acq_members])

    # Create a DataFrame
    columns = [
        "event_date", "member_id", "mkt_channel", "campaign_name", "campaign_product",
          "conversion_product", "conversion_channel", "conversion_type"   
    ]

    return pd.DataFrame(event_data, columns=columns)





In [55]:
lta_evt_data_display = generate_lta_data(start_dt, end_dt, mbr_data, Paid_Display, 'Paid Display', campaign_names_chnl,campaign_products, conversion_channels)
lta_evt_data_search = generate_lta_data(start_dt, end_dt, mbr_data, Paid_Search, 'Paid Search', campaign_names_chnl,campaign_products, conversion_channels)
lta_evt_data_social = generate_lta_data(start_dt, end_dt, mbr_data, Paid_Social, 'Paid Social', campaign_names_chnl,campaign_products, conversion_channels)
lta_evt_data_email = generate_lta_data(start_dt, end_dt, mbr_data, Email, 'Email', campaign_names_chnl,campaign_products, conversion_channels)
lta_evt_data_dm = generate_lta_data(start_dt, end_dt, mbr_data, Direct_Mail, 'Direct Mail', campaign_names_chnl,campaign_products, conversion_channels)

lta_evt_data_display.head()

,event_date,member_id,mkt_channel,campaign_name,campaign_product,conversion_product,conversion_channel,conversion_type
0,2025-01-01,MzMwODQxOTY=,Paid Display,P&C_Auto Insurance_Paid Display_Campaign_1,Auto Insurance,Auto Insurance,Internet,quote_start
1,2025-01-01,Mjk5MDEzMDk=,Paid Display,P&C_Auto Insurance_Paid Display_Campaign_1,Auto Insurance,Auto Insurance,Internet,quote_start
2,2025-01-01,OTM0ODQ3Mzg=,Paid Display,P&C_Auto Insurance_Paid Display_Campaign_1,Auto Insurance,Auto Insurance,Internet,quote_start
3,2025-01-01,NTM4OTc1MDQ=,Paid Display,P&C_Auto Insurance_Paid Display_Campaign_1,Auto Insurance,Auto Insurance,Internet,quote_start
4,2025-01-01,NDgxOTI2Mzc=,Paid Display,P&C_Auto Insurance_Paid Display_Campaign_1,Auto Insurance,Auto Insurance,Internet,quote_start


In [56]:
len(lta_evt_data_display)

3644758

In [57]:
lta_evt_data_display.to_csv(base_path / "data" / "lta_evt_data_paid_display.csv", index=False)
lta_evt_data_search.to_csv(base_path / "data" / "lta_evt_data_paid_search.csv", index=False)
lta_evt_data_social.to_csv(base_path / "data" / "lta_evt_data_paid_social.csv", index=False)
lta_evt_data_email.to_csv(base_path / "data" / "lta_evt_data_email.csv", index=False)
lta_evt_data_dm.to_csv(base_path / "data" / "lta_evt_data_direct_mail.csv", index=False)